In [63]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import numpy as np
import scipy.sparse as scps
import scipy.sparse.linalg as ssl
import math
from numpy.linalg import det
from mpl_toolkits.mplot3d import Axes3D
from scipy.sparse import lil_matrix

In [64]:
def maillage(n):
#
# Une discretisation possible d'une EDP elliptique sur le domaine ]0,1[ x ]0,1[
# Le carre [0,1]x[0,1] est maille uniquement avec des triangles; 
# Les conditions limites sont de type Dirichlet uniquement   => neumann  =[];
#
# Entrees :
# n : nombre de points par cote du care => Npts points de discretisation au
# total
#
# Sorties :
# coordinates : matrice a deux colonnes. Chaque ligne contient les 
# coordonnes 2D d'un des points de la discretisation. Ces sommets seront 
# identifies a l'indice de la ligne correspondante dans la matrice
# coordinates.
# elements3 : matrice a trois colonnes. Chaque ligne contient les indices 
# des sommets d'un element triangle, dans le sens antihoraire. 
# dirichlet : vecteur colonne des indices des sommets de la frontiere de
# Dirichlet.
# neumann : matrice a deux colonnes. Chaque ligne contient les indices 
# des deux sommets d'une arete de la frontiere de Neumann.
# (neumann est vide sur cet exemple)
#
##################################################################################
    h=1/(n-1)
    npoin       = n*n ; 
    nelem       = 2*(n-1)*(n-1) ;
    coordinates = np.zeros((npoin,2)); 
    elements3   = (np.zeros((nelem,3))).astype(int) ;
    neumann     = [];
    dirichlet=(np.zeros((4*n-4,1))).astype(int)
    # Coordonnees et connectivites :
    e = -1 ; 
    p = -1 ;
    x=np.zeros((n+1,1))
    x[n,0]=1.
    for l in range (n+1):
        x[l,0]=l*h
    for j in range (n):
            for i in range(n):
                p = p + 1  
                coordinates[p,0] = x[i,0]  
                coordinates[p,1] = x[j,0] 
                if ((i != n-1) & (j != n-1)):
                    p1 = p
                    p2 = p1 + 1 
                    p3 = p1 + n 
                    p4 = p2 + n 
                    e = e + 1 
                    elements3[e,0] = p1 
                    elements3[e,1] = p2 
                    elements3[e,2] = p3 
                    e = e + 1
                    elements3[e,0] = p4 
                    elements3[e,1] = p3 
                    elements3[e,2] = p2 
    #Liste des sommets de la frontiere de Dirichlet:
    p=-1
    for j in range(n):
        p=p+1
        dirichlet[p,0] = j  
    for j in range(n*2-1,n*(n-1),n):
        p=p+1
        dirichlet[p,0] = j 
    for j in range(n*n-1,n*n-n-1,-1):
        p=p+1
        dirichlet[p,0] = j 
    for j in range(n*n-2*n,n-1,-n):
        p=p+1
        dirichlet[p,0] = j 

    return coordinates, elements3,dirichlet, neumann

In [65]:
def show(coordinates,u):
#
# Fonction d'affichage de la solution u sur le maillage defini par
# elements3, coordinates.
#
# Entrees:
# elements3 : matrice a trois colonnes contenant les elements triangles
# de la discretisation, identifies par les indices de leurs trois
# sommets.
# coordinates : matrice a deux colonnes contenant les coordonnes 2D des
# points de la discretisation.
# u : vecteur colonne de longueur egale au nombre de lignes de
# coordinates contenant les valeurs de la solution a afficher aux
# points de la discretisation.
#
# Sorties : Aucune, mais la fonction doit s'afficher dans une figure.
##########################################################################
    ax= plt.figure().add_subplot(projection='3d')
    ax.plot_trisurf(coordinates[:,0],coordinates[:,1],u,linewidth=0.2,antialiased=True)
    plt.show()

**Partie I : maillage triangulaire et conditions de Dirichlet**

In [66]:

'''
def compute_MTA():
    # Construction de la matrice de raideur élémentaire MTA relative à un élément triangle
    
    # Coordonnées des sommets de l'élément triangle dans le plan (x,y)
    x = np.array([0, 1, 0])
    y = np.array([0, 0, 1])
    
    # Calcul des gradients des fonctions de base de l'élément triangle
    J = np.array([[1, 1, 1], [x[0], x[1], x[2]], [y[0], y[1], y[2]]])
    detJ = np.linalg.det(J)
    invJ = np.linalg.inv(J)
    G = np.matmul(invJ, np.array([[0, 1, 0], [0, 0, 1], [1, 1, 1]]))
    
    # Calcul de la matrice de raideur élémentaire MTA
    MTA = np.matmul(np.transpose(G), np.matmul(np.array([[1, -1, 0], [-1, 1, 0], [0, 0, 0]]), G)) * detJ / 2
    
    return MTA '''

def assemble_A(coordinates, elements3,dirichlet,ud,f):
    # Assemblage de la matrice A dans le cas d'un maillage constitué uniquement d'éléments triangles
    
    n = coordinates.shape[0]
    A = lil_matrix((n, n))
    b = np.zeros((n, 1))

    for e in elements3:
        p1, p2, p3 = e
        x1, y1 = coordinates[p1]
        x2, y2 = coordinates[p2]
        x3, y3 = coordinates[p3]
        detJ = abs((x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1))
        area = detJ / 2
        B = 1 / (2 * area) * np.array([[y2 - y3, y3 - y1, y1 - y2], [x3 - x2, x1 - x3, x2 - x1]])
        Ke = area * np.dot(B.T, B)
        for i, pi in enumerate(e):
            for j, pj in enumerate(e):
                A[pi, pj] += Ke[i, j]
            b[pi] += area * f(coordinates[pi][0],coordinates[pi][1])
    for p in dirichlet:
        A[p] = 0
        A[p, p] = 1
        b[p] = ud[coordinates[p]]
    A = A.tocsr()                  
        #############################################################
    return A,b

In [67]:
coordinates, elements3,dirichlet, neumann = maillage(10)   
f = np.cos(np.linspace(0.,np.pi,100))
ud = np.ones((1,100))
npoin = len(coordinates)
nelem = len(elements3)
A = np.zeros((npoin, npoin))
    
for t in elements3:
        grads = np.empty((3,), dtype=object)
        p1,p2,p3 = coordinates[t[0]],coordinates[t[1]],coordinates[t[2]]
        alpha = (p2[0]-p1[0])*(p3[1]-p1[1]) - (p3[0] - p1[0])*(p2[1]- p1[1])
        for i in range(3):
            grads[i] = (1/alpha)*np.array([coordinates[t[(i+1)%3]][1]-coordinates[t[(i+2)%3]][1],coordinates[t[(i+2)%3]][0]-coordinates[t[(i+1)%3]][0]])

**Partie II : maillage mixte et ajoût des conditions de Neumann**

In [68]:
############################# Maillage mixte ################
e3=np.array([[1,2,12],[2,3,12],[3,4,14],[4,5,14],[2,15,3],[3,15,4]]).astype(int)
e4=np.array([[0,1,12,11],[11,12,13,10],[12,3,14,13],[10,13,8,9],[13,14,7,8],[14,5,6,7]]).astype(int)
dds=np.array([2,15,4,6,7,8,9,10,11,0]).astype(int)
nns=np.array([[4,5],[5,6],[0,1],[1,2]]).astype(int)
ccs=np.array([[0.,0.],[0.33333333333333,0],[0.53333333333333,0.],
                      [0.66666666666667,0.33333333333333],[1.,0.47],[1,0.66666666666667],
                     [1.,1.],[0.66666666666667,1.],[0.33333333333333,1.], [0.,1.],
                     [0.,0.66666666666667],[0.,0.33333333333333],[0.33333333333333,0.33333333333333],
                     [0.33333333333333,0.66666666666667],[0.66666666666667,0.66666666666667],[1.,0.]])

In [69]:
n = 15
N = n*n
coordinates, elements3,dirichlet, neumann = maillage(n)
f = lambda x,y : 0
A,b = assemble_A(coordinates,elements3,dirichlet,np.ones(N),f)
x = np.linalg.solve(A,b)
#print(A)
#print(b)
show(coordinates,x[:,0])

TypeError: 'numpy.ndarray' object is not callable

**Compléments  :  un nouveau terme dans l'EDP**